# Held-Karp LP Solver

In [11]:
import pulp

def solve_held_karp(vertices, edges):
    prob = pulp.LpProblem("Held-Karp_Relaxation", pulp.LpMinimize)
    x = pulp.LpVariable.dicts("x", [(i, j) for i, j, _ in edges], 
                              lowBound=0, upBound=1, cat=pulp.LpContinuous)

    prob += pulp.lpSum(c * x[i, j] for i, j, c in edges)

    for U in [set(vertices) - {v} for v in vertices]:
        prob += pulp.lpSum(x[i, j] for i, j, _ in edges if i in U and j not in U) >= 1
    
    for v in vertices:
        prob += pulp.lpSum(x[i, j] for i, j, _ in edges if i == v) == 1
        prob += pulp.lpSum(x[i, j] for i, j, _ in edges if j == v) == 1
    
    # Solve the LP
    prob.solve()
    
    # Extract results
    status = pulp.LpStatus[prob.status]
    total_cost = pulp.value(prob.objective)
    edge_solution = {(i, j): x[i, j].varValue for i, j, _ in edges}
    
    return status, total_cost, edge_solution

In [12]:
# Test cases
vertices = [0, 1, 2]
edges = [
    (0, 1, 2), (1, 2, 2), (2, 0, 2),
    (0, 2, 1), (2, 1, 1), (1, 0, 1)
]

status, total_cost, edge_solution = solve_held_karp(vertices, edges)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/chuck/miniforge3/envs/CS632/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/tf/kyhcn0rj15722n_6n2h54bcc0000gn/T/bae8ee8ea6ff4c68b871cf9d2c1d8d13-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/tf/kyhcn0rj15722n_6n2h54bcc0000gn/T/bae8ee8ea6ff4c68b871cf9d2c1d8d13-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 39 RHS
At line 49 BOUNDS
At line 56 ENDATA
Problem MODEL has 9 rows, 6 columns and 18 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-9) rows, 0 (-6) columns and 0 (-18) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 3
After Postsolve, objective 3, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 3 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed fro

In [13]:
# Check against networkx package
import networkx as nx
from networkx.algorithms.approximation.traveling_salesman import held_karp_ascent

G = nx.complete_graph(3, create_using=nx.DiGraph)
nx.set_edge_attributes(G, {(0, 1): 2, (1, 2): 2, (2, 0): 2, (0, 2): 1, (2, 1): 1, (1, 0): 1}, "weight")

opt_hk, z_star = held_karp_ascent(G, 'weight')
print(opt_hk)
print(z_star.edges)

3.0
[(1, 0), (2, 1), (0, 2)]


# Sample Spanning Tree

In [ ]:
support = list()
for (u, v), value_uv in edge_solution.items():
    if value_uv > 0:  
        value_vu = edge_solution.get((v, u), 0)
        support.add(set({i, j})) 
support

{frozenset({0, 1}), frozenset({0, 2}), frozenset({1, 2})}